# 📊 Part 3: Geometric Analysis & Visualization

This notebook performs deep geometric analysis of the Delta Observer's latent space and generates key visualizations.

## Key Questions

| Question | Metric | Our Finding |
|----------|--------|-------------|
| Can we decode semantics linearly? | R² | **0.9879** (Yes!) |
| Are semantics geometrically clustered? | Silhouette | **-0.02** (No!) |
| Was clustering ever present? | Trajectory | **Yes, transiently!** |

📄 **Paper:** [OSF MetaArXiv](https://doi.org/10.17605/OSF.IO/CNJTP)  
🔗 **Code:** [github.com/EntroMorphic/delta-observer](https://github.com/EntroMorphic/delta-observer)

---

## 📦 Setup

In [ ]:
# Install dependencies if needed (Colab)
import subprocess
import sys

def install_if_needed(package):
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

install_if_needed('umap-learn')
install_if_needed('scikit-learn')
install_if_needed('matplotlib')

print('✅ Dependencies ready!')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples, r2_score
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split
import os

# Plotting style
plt.style.use('default')
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Colors for consistency
COLORS = {
    'r2': '#2ecc71',
    'silhouette': '#e74c3c',
    'online': '#3498db',
    'posthoc': '#9b59b6',
    'pca': '#95a5a6'
}

print('✅ Imports complete!')

---

## 📂 Load Data

In [ ]:
# Clone repository if running in Colab
repo_dir = 'delta-observer'
if not os.path.exists(repo_dir) and not os.path.exists('../data'):
    print('📥 Cloning delta-observer repository...')
    !git clone https://github.com/EntroMorphic/delta-observer.git
    print('✅ Repository cloned!')

# Smart path detection
latent_paths = [
    '../data/online_observer_latents.npz',
    'data/online_observer_latents.npz',
    'delta-observer/data/online_observer_latents.npz'
]

trajectory_paths = [
    '../data/online_observer_trajectory.npz',
    'data/online_observer_trajectory.npz',
    'delta-observer/data/online_observer_trajectory.npz'
]

latent_path = next((p for p in latent_paths if os.path.exists(p)), None)
trajectory_path = next((p for p in trajectory_paths if os.path.exists(p)), None)

figures_dir = '../figures' if os.path.exists('../figures') else 'figures'
os.makedirs(figures_dir, exist_ok=True)

print(f'📁 Latent data: {latent_path}')
print(f'📁 Trajectory data: {trajectory_path}')
print(f'📁 Figures directory: {figures_dir}')

In [ ]:
# Load data
latents_data = np.load(latent_path)
trajectory_data = np.load(trajectory_path)

latent_space = latents_data['latents']
carry_counts = latents_data['carry_counts']
mono_act = latents_data['mono_activations']
comp_act = latents_data['comp_activations']

snapshots = trajectory_data['snapshots']
epochs = trajectory_data['epochs']

print(f'\n📊 Data Summary:')
print(f'   Latent space: {latent_space.shape}')
print(f'   Carry counts: {np.bincount(carry_counts)}')
print(f'   Trajectory: {snapshots.shape} ({len(epochs)} snapshots)')
print('\n✅ Data loaded successfully!')

---

## 📈 Analysis 1: Linear Accessibility (R²)

**Question:** Can we predict carry count from latent space using a linear model?

**Method:** Train Ridge regression, measure R² on test set.

In [ ]:
# Full data R²
reg_full = LinearRegression().fit(latent_space, carry_counts)
r2_full = r2_score(carry_counts, reg_full.predict(latent_space))

# Train/test split R²
X_train, X_test, y_train, y_test = train_test_split(
    latent_space, carry_counts, test_size=0.2, random_state=42
)
probe = Ridge(alpha=1.0)
probe.fit(X_train, y_train)
y_pred = probe.predict(X_test)
r2_test = r2_score(y_test, y_pred)

print('\n' + '='*60)
print('📈 LINEAR ACCESSIBILITY ANALYSIS')
print('='*60)
print(f'\n   R² (full data):    {r2_full:.4f}')
print(f'   R² (test split):   {r2_test:.4f}')
print(f'\n   → {r2_full:.1%} of carry count variance explained linearly!')
print('='*60)

In [ ]:
# 🎨 Visualize linear probe
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Prediction scatter
ax1 = axes[0]
ax1.scatter(y_test, y_pred, alpha=0.6, c=COLORS['r2'], s=50)
ax1.plot([0, 4], [0, 4], 'k--', linewidth=2, label='Perfect')
ax1.set_xlabel('True Carry Count', fontsize=12)
ax1.set_ylabel('Predicted Carry Count', fontsize=12)
ax1.set_title(f'📊 Linear Probe (R² = {r2_test:.4f})', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Residuals
ax2 = axes[1]
residuals = y_test - y_pred
ax2.scatter(y_pred, residuals, alpha=0.6, c=COLORS['r2'], s=50)
ax2.axhline(0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Predicted', fontsize=12)
ax2.set_ylabel('Residual', fontsize=12)
ax2.set_title('📉 Residual Plot', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Coefficient importance
ax3 = axes[2]
coef_importance = np.abs(reg_full.coef_)
ax3.bar(range(len(coef_importance)), coef_importance, color=COLORS['r2'], alpha=0.7)
ax3.set_xlabel('Latent Dimension', fontsize=12)
ax3.set_ylabel('|Coefficient|', fontsize=12)
ax3.set_title('📊 Feature Importance', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'linear_accessibility_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## 📉 Analysis 2: Geometric Clustering (Silhouette)

**Question:** Are points with similar carry counts clustered together in space?

**Method:** Compute Silhouette score using carry count as cluster labels.

In [ ]:
# Compute silhouette scores
sil_global = silhouette_score(latent_space, carry_counts)
sil_samples = silhouette_samples(latent_space, carry_counts)

# Per-class silhouette
sil_per_class = {}
for c in range(5):
    mask = carry_counts == c
    if mask.sum() > 0:
        sil_per_class[c] = sil_samples[mask].mean()

print('\n' + '='*60)
print('📉 GEOMETRIC CLUSTERING ANALYSIS')
print('='*60)
print(f'\n   Global Silhouette: {sil_global:.4f}')
print(f'\n   Per-class Silhouette:')
for c, s in sil_per_class.items():
    print(f'      Carry {c}: {s:.4f}')
print(f'\n   → Near-zero indicates NO geometric clustering!')
print('='*60)

In [ ]:
# 🎨 Visualize silhouette analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 1. Silhouette by sample
ax1 = axes[0]
y_lower = 10
for i in range(5):
    ith_values = sil_samples[carry_counts == i]
    ith_values.sort()
    size = len(ith_values)
    y_upper = y_lower + size
    
    color = plt.cm.viridis(float(i) / 5)
    ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_values,
                      facecolor=color, edgecolor=color, alpha=0.7)
    ax1.text(-0.05, y_lower + 0.5 * size, f'{i}', fontsize=10, fontweight='bold')
    y_lower = y_upper + 10

ax1.axvline(sil_global, color='red', linestyle='--', linewidth=2, label=f'Mean: {sil_global:.3f}')
ax1.axvline(0, color='black', linestyle='-', linewidth=1)
ax1.set_xlabel('Silhouette Score', fontsize=12)
ax1.set_ylabel('Carry Count', fontsize=12)
ax1.set_title('📊 Silhouette by Sample', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Per-class comparison
ax2 = axes[1]
classes = list(sil_per_class.keys())
values = list(sil_per_class.values())
colors = [plt.cm.viridis(c/5) for c in classes]
bars = ax2.bar(classes, values, color=colors, alpha=0.7, edgecolor='black')
ax2.axhline(0, color='gray', linestyle='--', linewidth=1)
ax2.axhline(sil_global, color='red', linestyle='--', linewidth=2, label=f'Global: {sil_global:.3f}')
ax2.set_xlabel('Carry Count', fontsize=12)
ax2.set_ylabel('Mean Silhouette', fontsize=12)
ax2.set_title('📊 Silhouette by Class', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Add value labels
for bar, val in zip(bars, values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{val:.3f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'silhouette_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## 🌟 The Paradox: Accessibility Without Clustering

**High R² + Low Silhouette = Continuous Semantic Gradients**

In [ ]:
# 🎨 The paradox visualization
fig, ax = plt.subplots(figsize=(10, 7))

metrics = ['Linear Accessibility\n(R²)', 'Geometric Clustering\n(Silhouette)']
values = [r2_full, max(0, sil_global)]
colors_bar = [COLORS['r2'], COLORS['silhouette']]

bars = ax.barh(metrics, values, color=colors_bar, alpha=0.8, height=0.5, edgecolor='black', linewidth=2)

ax.set_xlim(0, 1.1)
ax.set_xlabel('Score', fontsize=12)
ax.set_title('🎯 The Accessibility-Clustering Paradox', fontsize=14, fontweight='bold')

# Value labels
ax.text(r2_full + 0.02, 0, f'{r2_full:.4f}', va='center', fontsize=14, fontweight='bold', color=COLORS['r2'])
ax.text(max(0.02, sil_global) + 0.02, 1, f'{sil_global:.4f}', va='center', fontsize=14, fontweight='bold', color=COLORS['silhouette'])

# Threshold lines
ax.axvline(0.5, color='gray', linestyle='--', alpha=0.5)

# Insight box
insight = ('KEY FINDING:\n\n'
           f'R² = {r2_full:.4f} → HIGH linear accessibility\n'
           f'Silhouette = {sil_global:.4f} → NO clustering\n\n'
           'Semantics exist as CONTINUOUS\n'
           'GRADIENTS, not discrete clusters!')
props = dict(boxstyle='round', facecolor='lightyellow', alpha=0.9, edgecolor='orange')
ax.text(0.55, 0.5, insight, transform=ax.transAxes, fontsize=11,
        verticalalignment='center', bbox=props)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'accessibility_clustering_paradox.png'), dpi=200, bbox_inches='tight')
plt.show()

---

## ⏳ Analysis 3: Transient Clustering

**The most surprising discovery:** Clustering isn't absent—it's *transient*!

In [ ]:
# Compute trajectory metrics
r2_trajectory = []
sil_trajectory = []

for i, epoch in enumerate(epochs):
    z = snapshots[i]
    
    # R²
    reg = LinearRegression().fit(z, carry_counts)
    r2_trajectory.append(r2_score(carry_counts, reg.predict(z)))
    
    # Silhouette
    try:
        sil_trajectory.append(silhouette_score(z, carry_counts))
    except:
        sil_trajectory.append(0)

r2_trajectory = np.array(r2_trajectory)
sil_trajectory = np.array(sil_trajectory)

# Find peak clustering
peak_idx = np.argmax(sil_trajectory)
peak_epoch = epochs[peak_idx]
peak_sil = sil_trajectory[peak_idx]

print('\n' + '='*60)
print('⏳ TRANSIENT CLUSTERING ANALYSIS')
print('='*60)
print(f'\n   Trajectory: {len(epochs)} snapshots ({epochs[0]} to {epochs[-1]} epochs)')
print(f'\n   R² range: {r2_trajectory.min():.4f} to {r2_trajectory.max():.4f}')
print(f'   Silhouette range: {sil_trajectory.min():.4f} to {sil_trajectory.max():.4f}')
print(f'\n   🔥 Peak clustering: Silhouette = {peak_sil:.4f} at epoch {peak_epoch}')
print(f'   📉 Final clustering: Silhouette = {sil_trajectory[-1]:.4f}')
print(f'\n   → Clustering EMERGES then DISSOLVES!')
print('='*60)

In [ ]:
# 🎨 Transient clustering visualization
fig, ax1 = plt.subplots(figsize=(12, 6))

# R² on left axis
ax1.set_xlabel('Training Epoch', fontsize=12)
ax1.set_ylabel('R² (Linear Accessibility)', color=COLORS['r2'], fontsize=12)
line1, = ax1.plot(epochs, r2_trajectory, color=COLORS['r2'], linewidth=2.5,
                  marker='o', markersize=4, label='R² (Accessibility)')
ax1.tick_params(axis='y', labelcolor=COLORS['r2'])
ax1.set_ylim(0, 1.05)
ax1.axhline(0.9, color=COLORS['r2'], linestyle='--', alpha=0.3)

# Silhouette on right axis
ax2 = ax1.twinx()
ax2.set_ylabel('Silhouette (Clustering)', color=COLORS['silhouette'], fontsize=12)
line2, = ax2.plot(epochs, sil_trajectory, color=COLORS['silhouette'], linewidth=2.5,
                  marker='s', markersize=4, label='Silhouette (Clustering)')
ax2.tick_params(axis='y', labelcolor=COLORS['silhouette'])
ax2.set_ylim(-0.1, 0.5)
ax2.axhline(0, color=COLORS['silhouette'], linestyle='--', alpha=0.3)

# Annotate peak
ax2.annotate(f'Peak: {peak_sil:.2f}\n(epoch {peak_epoch})',
             xy=(peak_epoch, peak_sil),
             xytext=(peak_epoch + 30, peak_sil + 0.08),
             fontsize=11, fontweight='bold',
             arrowprops=dict(arrowstyle='->', color=COLORS['silhouette'], lw=2),
             color=COLORS['silhouette'])

# Title and legend
ax1.set_title('⏳ Transient Clustering: Scaffolding Emerges Then Dissolves',
              fontsize=14, fontweight='bold')

lines = [line1, line2]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='center right', fontsize=10)

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'figure5_training_curves.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 🎨 Latent space evolution
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Select key epochs: early, peak, final
key_epochs = [0, peak_idx, -1]
titles = ['📍 Early (Random)', f'🔥 Peak (Epoch {peak_epoch})', '📍 Final (Converged)']

for ax, idx, title in zip(axes, key_epochs, titles):
    z = snapshots[idx]
    
    # PCA projection
    pca = PCA(n_components=2)
    z_2d = pca.fit_transform(z)
    
    # Compute metrics
    reg = LinearRegression().fit(z, carry_counts)
    r2 = r2_score(carry_counts, reg.predict(z))
    sil = sil_trajectory[idx]
    
    scatter = ax.scatter(z_2d[:, 0], z_2d[:, 1], c=carry_counts, cmap='viridis',
                         s=30, alpha=0.7, edgecolors='white', linewidth=0.3)
    
    ax.set_title(f'{title}\nR²={r2:.3f}, Sil={sil:.3f}', fontsize=12, fontweight='bold')
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')

# Add colorbar on the right side without overlapping
fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
fig.colorbar(scatter, cax=cbar_ax, label='Carry Count')

plt.suptitle('📊 Latent Space Evolution During Training', fontsize=14, fontweight='bold', y=1.02)
plt.savefig(os.path.join(figures_dir, 'latent_evolution.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## 🎨 Latent Space Visualizations

In [ ]:
# UMAP and PCA comparison
try:
    from umap import UMAP
    umap_reducer = UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
    latent_umap = umap_reducer.fit_transform(latent_space)
    has_umap = True
except ImportError:
    has_umap = False
    print('⚠️ UMAP not available, using PCA only')

pca_reducer = PCA(n_components=2)
latent_pca = pca_reducer.fit_transform(latent_space)
explained_var = pca_reducer.explained_variance_ratio_

print(f'PCA variance explained: {explained_var[0]:.1%} + {explained_var[1]:.1%} = {sum(explained_var[:2]):.1%}')

In [ ]:
# 🎨 Comprehensive latent space visualization
if has_umap:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # PCA
    ax1 = axes[0]
    scatter1 = ax1.scatter(latent_pca[:, 0], latent_pca[:, 1], c=carry_counts, cmap='viridis',
                           s=40, alpha=0.7, edgecolors='white', linewidth=0.3)
    ax1.set_title(f'PCA Projection\n(Var: {sum(explained_var[:2]):.1%})', fontsize=13, fontweight='bold')
    ax1.set_xlabel(f'PC1 ({explained_var[0]:.1%})')
    ax1.set_ylabel(f'PC2 ({explained_var[1]:.1%})')
    
    # UMAP
    ax2 = axes[1]
    scatter2 = ax2.scatter(latent_umap[:, 0], latent_umap[:, 1], c=carry_counts, cmap='viridis',
                           s=40, alpha=0.7, edgecolors='white', linewidth=0.3)
    ax2.set_title('UMAP Projection', fontsize=13, fontweight='bold')
    ax2.set_xlabel('UMAP 1')
    ax2.set_ylabel('UMAP 2')
    
    # Add colorbar on the right side without overlapping
    fig.subplots_adjust(right=0.88)
    cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
    fig.colorbar(scatter2, cax=cbar_ax, label='Carry Count')
else:
    fig, ax = plt.subplots(figsize=(10, 8))
    scatter = ax.scatter(latent_pca[:, 0], latent_pca[:, 1], c=carry_counts, cmap='viridis',
                         s=50, alpha=0.7, edgecolors='white', linewidth=0.5)
    ax.set_title(f'PCA Projection (Var: {sum(explained_var[:2]):.1%})', fontsize=14, fontweight='bold')
    ax.set_xlabel(f'PC1 ({explained_var[0]:.1%})')
    ax.set_ylabel(f'PC2 ({explained_var[1]:.1%})')
    plt.colorbar(scatter, ax=ax, label='Carry Count')

plt.suptitle('🔬 Online Delta Observer Latent Space', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'figure2_delta_latent_space.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## 🏆 Method Comparison

In [ ]:
# Compute PCA baseline
combined_act = np.concatenate([mono_act, comp_act], axis=1)
pca_baseline = PCA(n_components=16, random_state=42)
z_pca = pca_baseline.fit_transform(combined_act)

reg_pca = LinearRegression().fit(z_pca, carry_counts)
r2_pca = r2_score(carry_counts, reg_pca.predict(z_pca))

# Results
methods = ['Online\nObserver', 'Post-hoc\nObserver', 'PCA\nBaseline']
r2_values = [r2_full, 0.9505, r2_pca]  # Post-hoc value from paper
colors_methods = [COLORS['online'], COLORS['posthoc'], COLORS['pca']]

print('\n' + '='*60)
print('🏆 METHOD COMPARISON')
print('='*60)
for m, r in zip(methods, r2_values):
    print(f'   {m.replace(chr(10), " "):20} R² = {r:.4f}')
print(f'\n   Δ Online vs PCA: +{(r2_full - r2_pca) / r2_pca * 100:.1f}%')
print('='*60)

In [ ]:
# 🎨 Method comparison visualization
fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(methods, r2_values, color=colors_methods, edgecolor='black', linewidth=2, alpha=0.8)

# Value labels
for bar, val in zip(bars, r2_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.003,
            f'{val:.4f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

# Delta annotation
ax.annotate('', xy=(0, r2_full), xytext=(2, r2_pca),
            arrowprops=dict(arrowstyle='<->', color='black', lw=2))
delta = (r2_full - r2_pca) / r2_pca * 100
ax.text(1, 0.965, f'+{delta:.1f}%', ha='center', fontsize=14, fontweight='bold', color='green')

ax.set_ylabel('R² (Linear Accessibility)', fontsize=12)
ax.set_title('🏆 Method Comparison: Online Observation Wins', fontsize=14, fontweight='bold')
ax.set_ylim(0.93, 1.01)
ax.axhline(y=r2_pca, color='gray', linestyle='--', alpha=0.5)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(figures_dir, 'figure_method_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

---

## 📝 Summary

In [ ]:
print('\n' + '='*70)
print('📊 DELTA OBSERVER: ANALYSIS SUMMARY')
print('='*70)

print('\n📈 LINEAR ACCESSIBILITY')
print(f'   R² = {r2_full:.4f}')
print('   → Semantic information is LINEARLY ACCESSIBLE')

print('\n📉 GEOMETRIC CLUSTERING')
print(f'   Silhouette = {sil_global:.4f}')
print('   → NO geometric clustering in final state')

print('\n⏳ TRANSIENT CLUSTERING')
print(f'   Peak: Silhouette = {peak_sil:.4f} at epoch {peak_epoch}')
print(f'   Final: Silhouette = {sil_trajectory[-1]:.4f}')
print('   → Clustering is SCAFFOLDING, not STRUCTURE!')

print('\n🏆 METHOD COMPARISON')
print(f'   Online Observer: R² = {r2_full:.4f}')
print(f'   PCA Baseline:    R² = {r2_pca:.4f}')
print(f'   Improvement:     +{delta:.1f}%')

print('\n' + '='*70)
print('KEY INSIGHT: The semantic primitive is in the')
print('             LEARNING TRAJECTORY, not the final state!')
print('='*70)

print('\n📁 Figures saved:')
for f in os.listdir(figures_dir):
    if f.endswith('.png'):
        print(f'   - {f}')

---

## 🚀 Next Steps

For complete end-to-end reproduction, see **`99_full_reproduction.ipynb`**.

| Notebook | Description | Colab |
|----------|-------------|-------|
| **00_quickstart_demo** | Quick demo | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EntroMorphic/delta-observer/blob/main/notebooks/00_quickstart_demo.ipynb) |
| **99_full_reproduction** | Complete pipeline | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EntroMorphic/delta-observer/blob/main/notebooks/99_full_reproduction.ipynb) |

---

**For Science!** 🔬🌊